In [208]:
import pandas as pd
import numpy as np
import pymc as pm

## Speed
Our response will be predicted get off time
    - Need to decide how we want to do this
    - Option 1 is amount of time to get to a certain distance (Say 1.5 yards)
    - Option 2 would be predicted yards given a cetain amount of time
For predictors:
    - Height/weight of the tackle
    - Quarter
    - Whether it's an obvious passing situation
    - Left vs right tackle?

## Power
Our response here is some variation of percentage of the distance to the QB given up once engaged with the defensive end
For predictors:
    - Height/weight of both the tackle and the end
    - Obvious passing situation
    - Time to throw for the QB
    - Quarter

In [209]:
model_data = pd.read_csv('https://media.githubusercontent.com/media/cnickol26/BigDataBowl2023/main/clustered.csv')

In [210]:
model_data.columns

Index(['uniqueplayId', 'QB_Flip', 'rushType', 'Power_prob', 'Speed_prob',
       'football_x', 'football_y', 'gameId', 'playId', 'frameId', 'time',
       'playDirection', 'event', 'ball_snap_frame', 'end_frame',
       'play_length_frames', 'Right_end', 'Left_end', 'QB_nflId',
       'QB_jerseyNumber', 'QB_team', 'QB_x', 'QB_y', 'QB_s', 'QB_a', 'QB_dis',
       'QB_o', 'QB_dir', 'QB_pff_role', 'QB_pff_positionLinedUp',
       'QB_pff_nflIdBlockedPlayer', 'QB_pff_blockType', 'QB_new_x', 'QB_new_y',
       'ball_nflId', 'ball_jerseyNumber', 'ball_team', 'ball_x', 'ball_y',
       'ball_s', 'ball_a', 'ball_dis', 'ball_o', 'ball_dir', 'ball_pff_role',
       'ball_pff_positionLinedUp', 'ball_pff_nflIdBlockedPlayer',
       'ball_pff_blockType', 'ball_new_x', 'ball_new_y', 'ball_Flip',
       'Tackle_nflId', 'Tackle_jerseyNumber', 'Tackle_team', 'Tackle_x',
       'Tackle_y', 'Tackle_s', 'Tackle_a', 'Tackle_dis', 'Tackle_o',
       'Tackle_dir', 'Tackle_pff_role', 'Tackle_pff_positionLined

In [211]:
model_data2 = model_data[['uniqueplayId','QB_Flip','rushType','Power_prob','Speed_prob',
            'football_x', 'football_y', 'gameId', 'playId', 'frameId', 'time',
       'event','play_length_frames','QB_s', 'QB_a', 'QB_dis',
       'QB_o', 'QB_dir', 'QB_pff_role', 'QB_pff_positionLinedUp','QB_new_x', 'QB_new_y',
            'ball_new_x', 'ball_new_y', 'ball_Flip','ball_snap_frame',
       'Tackle_nflId', 'Tackle_s', 'Tackle_a', 'Tackle_dis', 'Tackle_o',
       'Tackle_dir', 'Tackle_pff_role', 'Tackle_pff_positionLinedUp',
       'Tackle_pff_nflIdBlockedPlayer', 'Tackle_pff_blockType', 'Tackle_new_x',
       'Tackle_new_y', 'Tackle_Flip', 'End_nflId', 'End_s', 'End_a', 'End_dis', 'End_o',
       'End_dir', 'End_pff_role', 'End_pff_positionLinedUp', 'End_new_x',
       'End_new_y', 'End_Flip', 'tackle_end_dist', 'tackle_end_facing']]

In [212]:
players = pd.read_csv('https://media.githubusercontent.com/media/cnickol26/BigDataBowl2023/main/nfl-big-data-bowl-2023/players.csv')

In [213]:
players = players[['height', 'weight', 'nflId','displayName']]

In [214]:
model_data3 = model_data2.merge(players, left_on = 'Tackle_nflId', right_on = 'nflId', how = 'left')

In [215]:
model_data4 = model_data3.rename(columns = {'height':'tackle_height','weight':'tackle_weight','displayName':'tackle_name'}).drop('nflId', axis = 1)

In [216]:
model_data5 = model_data4.merge(players, left_on = 'End_nflId', right_on = 'nflId', how = 'left')

In [217]:
model_data6 = model_data5.rename(columns = {'height':'end_height','weight':'end_weight','displayName':'end_name'}).drop('nflId', axis = 1)

In [218]:
plays = pd.read_csv('https://media.githubusercontent.com/media/cnickol26/BigDataBowl2023/main/nfl-big-data-bowl-2023/plays.csv')

In [219]:
plays['obvious_pass'] = ((plays['down'] == 2) & (plays['yardsToGo'] > 10)) | ((plays['down'] == 3) & (plays['yardsToGo'] > 4)) | ((plays['down'] == 4) & (plays['yardsToGo'] > 2))

In [220]:
plays['obvious_pass'] = plays['obvious_pass'].astype(int)

In [221]:
plays = plays[['gameId','playId','quarter','obvious_pass']]

In [222]:
plays

,gameId,playId,quarter,obvious_pass
0,2021090900,97,1,0
1,2021090900,137,1,0
2,2021090900,187,1,0
3,2021090900,282,1,0
4,2021090900,349,1,1
...,...,...,...,...
8553,2021110100,4310,4,1
8554,2021110100,4363,4,0
8555,2021110100,4392,4,0
8556,2021110100,4411,4,1


In [223]:
model_data7 = model_data6.merge(plays, on = ['gameId','playId'], how = 'left')

In [224]:
model_data7['tackle_end_engaged'] = (model_data7['tackle_end_facing'] <= 40) & (model_data7['tackle_end_dist'] <= 1)

In [225]:
model_data7['tackle_end_engaged'] = model_data7['tackle_end_engaged'].astype(int)

In [226]:
model_data7['tackle_end_engaged'].value_counts()

0    224075
1     61595
Name: tackle_end_engaged, dtype: int64

In [227]:
def dist(x_1, y_1, x_2, y_2):
    return np.sqrt(np.sum([(x_1-x_2)**2, (y_1-y_2)**2], axis=0))

In [228]:
model_data7['tackle_qb_dist'] = dist(model_data7['Tackle_new_x'], model_data7['Tackle_new_y'], model_data7['QB_new_x'], model_data7['QB_new_y'])

In [229]:
model_data7['end_qb_dist'] = dist(model_data7['End_new_x'], model_data7['End_new_y'], model_data7['QB_new_x'], model_data7['QB_new_y'])

In [230]:
speed = model_data7[model_data7['rushType'] == 'Speed']
speed

,uniqueplayId,QB_Flip,rushType,Power_prob,Speed_prob,football_x,football_y,gameId,playId,frameId,...,tackle_weight,tackle_name,end_height,end_weight,end_name,quarter,obvious_pass,tackle_end_engaged,tackle_qb_dist,end_qb_dist
0,202109090097,1.0,Speed,0.08467,0.91533,41.56,23.92,2021090900,97,6,...,322,Tristan Wirfs,6-3,245,Micah Parsons,1,0,0,4.554492,6.473832
1,202109090097,1.0,Speed,0.08467,0.91533,41.56,23.92,2021090900,97,7,...,322,Tristan Wirfs,6-3,245,Micah Parsons,1,0,0,4.610878,6.481790
2,202109090097,1.0,Speed,0.08467,0.91533,41.56,23.92,2021090900,97,8,...,322,Tristan Wirfs,6-3,245,Micah Parsons,1,0,0,4.651021,6.451364
3,202109090097,1.0,Speed,0.08467,0.91533,41.56,23.92,2021090900,97,9,...,322,Tristan Wirfs,6-3,245,Micah Parsons,1,0,0,4.667987,6.361549
4,202109090097,1.0,Speed,0.08467,0.91533,41.56,23.92,2021090900,97,10,...,322,Tristan Wirfs,6-3,245,Micah Parsons,1,0,0,4.714106,6.192205
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
285594,20211017071173,0.0,Speed,NaN,NaN,57.90,24.08,2021101707,1173,27,...,305,Charles Leno,6-0,232,Nick Bolton,2,0,0,4.893107,6.758913
285595,20211017071173,0.0,Speed,NaN,NaN,57.90,24.08,2021101707,1173,28,...,305,Charles Leno,6-0,232,Nick Bolton,2,0,0,4.787839,6.807437
285596,20211017071173,0.0,Speed,NaN,NaN,57.90,24.08,2021101707,1173,29,...,305,Charles Leno,6-0,232,Nick Bolton,2,0,0,4.674409,6.866513
285597,20211017071173,0.0,Speed,NaN,NaN,57.90,24.08,2021101707,1173,30,...,305,Charles Leno,6-0,232,Nick Bolton,2,0,0,4.555052,6.941823


In [231]:
power = model_data7[model_data7['rushType'] == 'Power']
power_filt = power[(power['tackle_qb_dist'] < power['end_qb_dist'])]
power_anti = power[(power['tackle_qb_dist'] >= power['end_qb_dist'])]

In [232]:
power_anti.groupby(['uniqueplayId', 'QB_Flip']).first().reset_index()

,uniqueplayId,QB_Flip,rushType,Power_prob,Speed_prob,football_x,football_y,gameId,playId,frameId,...,tackle_weight,tackle_name,end_height,end_weight,end_name,quarter,obvious_pass,tackle_end_engaged,tackle_qb_dist,end_qb_dist
0,202109120163,1.0,Power,0.96785,0.03215,34.03,23.76,2021091201,63,23,...,330,Daryl Williams,6-4,252,T.J. Watt,1,0,1,5.415773,5.259553
1,202109120676,0.0,Power,0.91197,0.08803,84.97,23.55,2021091206,76,22,...,325,Julien Davenport,6-2,270,Kerry Hyder,1,0,1,4.795873,4.752431
2,202109190595,0.0,Power,0.98233,0.01767,33.13,29.96,2021091905,95,36,...,310,Austin Jackson,6-2,254,Jerry Hughes,1,1,0,2.837816,2.781726
3,202109190975,0.0,Power,0.74182,0.25818,74.23,29.79,2021091909,75,29,...,313,Rashod Hill,6-5,288,J.J. Watt,1,0,0,6.041523,6.039106
4,202109191054,0.0,Power,0.97524,0.02476,35.05,26.58,2021091910,54,35,...,338,Donovan Smith,6-3,255,Dante Fowler,1,0,0,4.460773,4.339551
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1467,20211025002214,0.0,Power,0.98339,0.01661,76.97,29.55,2021102500,2214,29,...,304,Terron Armstead,6-3,265,Benson Mayowa,3,0,0,5.383995,5.252361
1468,20211025002348,1.0,Power,0.96204,0.03796,27.20,29.88,2021102500,2348,31,...,324,Brandon Shell,6-6,265,Marcus Davenport,3,0,0,1.254472,1.054372
1469,20211025002934,1.0,Power,0.92624,0.07376,77.51,24.50,2021102500,2934,33,...,324,Brandon Shell,6-4,287,Cameron Jordan,3,1,0,3.638035,3.532987
1470,20211025003260,0.0,Power,0.83014,0.16986,86.05,24.02,2021102500,3260,29,...,315,Duane Brown,6-6,265,Marcus Davenport,4,1,1,4.968149,4.824106


In [245]:
model_data7[model_data7['event']=='line_set']

,uniqueplayId,QB_Flip,rushType,Power_prob,Speed_prob,football_x,football_y,gameId,playId,frameId,...,tackle_weight,tackle_name,end_height,end_weight,end_name,quarter,obvious_pass,tackle_end_engaged,tackle_qb_dist,end_qb_dist
61535,2021102408785,0.0,Power,0.98455,0.01545,69.71,23.88,2021102408,785,8,...,315,Geron Christian,6-5,288,J.J. Watt,1,1,0,4.900500,7.253192
61559,2021102408785,1.0,Power,0.98510,0.01490,69.71,23.88,2021102408,785,8,...,315,Charlie Heck,6-3,260,Markus Golden,1,1,0,4.442454,7.508562
67522,20210912003776,0.0,Power,0.95206,0.04794,87.25,30.00,2021091200,3776,9,...,309,Jake Matthews,6-3,305,Marlon Tuipulotu,4,0,0,3.189545,3.168122
67560,20210912003776,1.0,Speed,0.02491,0.97509,87.25,30.00,2021091200,3776,9,...,306,Kaleb McGary,6-4,265,Ryan Kerrigan,4,0,0,3.140143,6.005939
128291,20210920002483,0.0,Speed,0.10072,0.89928,62.02,29.77,2021092000,2483,6,...,310,Elgton Jenkins,6-3,250,Charles Harris,3,1,0,3.301227,5.357649
128325,20210920002483,1.0,Power,0.91964,0.08036,62.02,29.77,2021092000,2483,6,...,310,Billy Turner,6-4,263,Romeo Okwara,3,1,0,3.450232,6.077911


In [246]:
model_data7[model_data7['uniqueplayId']==2021102408785][['frameId', 'event']]

,frameId,event
61533,6,ball_snap
61534,7,autoevent_ballsnap
61535,8,line_set
61536,9,None
61537,10,None
61538,11,None
61539,12,None
61540,13,None
61541,14,None
61542,15,None


In [244]:
model_data7[model_data7['uniqueplayId']==20211010043129][['frameId', 'event']]

,frameId,event
207303,7,ball_snap
207304,8,None
207305,9,None
207306,10,None
207307,11,None
207308,12,None
207309,13,None
207310,14,None
207311,15,None
207312,16,None


In [240]:
model_data7['event'].unique()

array(['ball_snap', 'None', 'autoevent_passforward', 'pass_forward',
       'autoevent_ballsnap', 'huddle_break_offense', 'pass_arrived',
       'play_action', 'qb_sack', 'qb_strip_sack', 'pass_tipped',
       'first_contact', 'autoevent_passinterrupted', 'handoff',
       'pass_outcome_incomplete', 'pass_outcome_caught', 'shift',
       'man_in_motion', 'fumble', 'line_set', 'fumble_offense_recovered',
       'run', 'dropped_pass'], dtype=object)

In [239]:
power[(power['uniqueplayId']== 202109190595)&(power['QB_Flip']==0)][['tackle_qb_dist', 'end_qb_dist', 
                                                                     'QB_new_x', 'QB_new_y', 
                                                                     'Tackle_new_x', 'Tackle_new_y', 
                                                                     'End_new_x', 'End_new_y', 'frameId', 'football_x',
                                                                     'football_y', 'event', 'tackle_name', 'end_name',
                                                                     'event']]

,tackle_qb_dist,end_qb_dist,QB_new_x,QB_new_y,Tackle_new_x,Tackle_new_y,End_new_x,End_new_y,frameId,football_x,football_y,event,tackle_name,end_name,event
902,4.589564,7.073083,-4.27,-0.12,-0.98,3.08,1.12,4.46,6,33.13,29.96,ball_snap,Austin Jackson,Jerry Hughes,ball_snap
903,4.610336,7.014449,-4.30,-0.14,-1.04,3.12,1.03,4.42,7,33.13,29.96,None,Austin Jackson,Jerry Hughes,None
904,4.624824,6.934219,-4.35,-0.15,-1.12,3.16,0.90,4.38,8,33.13,29.96,None,Austin Jackson,Jerry Hughes,None
905,4.653816,6.848562,-4.43,-0.17,-1.21,3.19,0.75,4.31,9,33.13,29.96,None,Austin Jackson,Jerry Hughes,None
906,4.675906,6.720595,-4.54,-0.19,-1.33,3.21,0.54,4.21,10,33.13,29.96,None,Austin Jackson,Jerry Hughes,None
907,4.711454,6.578526,-4.67,-0.22,-1.44,3.21,0.30,4.09,11,33.13,29.96,None,Austin Jackson,Jerry Hughes,None
908,4.732071,6.416829,-4.83,-0.25,-1.57,3.18,0.03,3.94,12,33.13,29.96,None,Austin Jackson,Jerry Hughes,None
909,4.766424,6.262787,-5.02,-0.28,-1.70,3.14,-0.26,3.79,13,33.13,29.96,None,Austin Jackson,Jerry Hughes,None
910,4.787118,6.116486,-5.24,-0.32,-1.85,3.06,-0.57,3.63,14,33.13,29.96,None,Austin Jackson,Jerry Hughes,None
911,4.802426,5.971641,-5.48,-0.35,-2.01,2.97,-0.89,3.47,15,33.13,29.96,None,Austin Jackson,Jerry Hughes,None


In [77]:
power_filt['rushType'].value_counts()

Power    178518
Name: rushType, dtype: int64

In [78]:
subset = power_filt[power_filt['tackle_end_engaged'] == 1]

In [79]:
subset2 = subset.groupby(['uniqueplayId','QB_Flip']).first().reset_index()

In [80]:
distance_when_engaged = subset2[['uniqueplayId','QB_Flip','tackle_qb_dist']]
distance_when_engaged

,uniqueplayId,QB_Flip,tackle_qb_dist
0,202109120163,0.0,3.927467
1,202109120163,1.0,5.853136
2,202109120578,1.0,4.997960
3,202109120676,0.0,4.894865
4,202109120676,1.0,3.900820
...,...,...,...
5547,20211025003660,1.0,5.355978
5548,20211025003904,0.0,3.735305
5549,20211025003904,1.0,4.439279
5550,20211025003926,0.0,4.787118


In [81]:
end_distance = power_filt.groupby(['uniqueplayId','QB_Flip']).last().reset_index()

In [82]:
end_distance = end_distance[['uniqueplayId','QB_Flip','tackle_qb_dist']]
end_distance

,uniqueplayId,QB_Flip,tackle_qb_dist
0,202109120163,0.0,2.438237
1,202109120163,1.0,5.594569
2,202109120288,1.0,6.659084
3,202109120578,1.0,2.407094
4,202109120676,0.0,4.827132
...,...,...,...
6218,20211025003660,1.0,3.138057
6219,20211025003904,0.0,2.595149
6220,20211025003904,1.0,4.460717
6221,20211025003926,0.0,8.396076


In [83]:
end_distance = end_distance.rename(columns = {'tackle_qb_dist': 'ending_distance'})

In [84]:
distance = distance_when_engaged.merge(end_distance, on = ['uniqueplayId', 'QB_Flip'], how = 'left')
distance

,uniqueplayId,QB_Flip,tackle_qb_dist,ending_distance
0,202109120163,0.0,3.927467,2.438237
1,202109120163,1.0,5.853136,5.594569
2,202109120578,1.0,4.997960,2.407094
3,202109120676,0.0,4.894865,4.827132
4,202109120676,1.0,3.900820,1.730665
...,...,...,...,...
5547,20211025003660,1.0,5.355978,3.138057
5548,20211025003904,0.0,3.735305,2.595149
5549,20211025003904,1.0,4.439279,4.460717
5550,20211025003926,0.0,4.787118,8.396076


In [85]:
distance['pct_given_up'] = (distance['tackle_qb_dist'] - distance['ending_distance']) / distance['tackle_qb_dist']
distance

,uniqueplayId,QB_Flip,tackle_qb_dist,ending_distance,pct_given_up
0,202109120163,0.0,3.927467,2.438237,0.379183
1,202109120163,1.0,5.853136,5.594569,0.044176
2,202109120578,1.0,4.997960,2.407094,0.518385
3,202109120676,0.0,4.894865,4.827132,0.013838
4,202109120676,1.0,3.900820,1.730665,0.556333
...,...,...,...,...,...
5547,20211025003660,1.0,5.355978,3.138057,0.414102
5548,20211025003904,0.0,3.735305,2.595149,0.305238
5549,20211025003904,1.0,4.439279,4.460717,-0.004829
5550,20211025003926,0.0,4.787118,8.396076,-0.753890


In [86]:
distance = distance.rename(columns = {'tackle_qb_dist' : 'starting_distance'})

In [87]:
model_data8 = power_filt.merge(distance, on = ['uniqueplayId','QB_Flip'], how = 'left')
model_data8

,uniqueplayId,QB_Flip,rushType,Power_prob,Speed_prob,football_x,football_y,gameId,playId,frameId,...,end_weight,end_name,quarter,obvious_pass,tackle_end_engaged,tackle_qb_dist,end_qb_dist,starting_distance,ending_distance,pct_given_up
0,202109120163,0.0,Power,0.98633,0.01367,34.03,23.76,2021091201,63,6,...,242,Alex Highsmith,1,0,0,4.820290,6.804594,3.927467,2.438237,0.379183
1,202109120163,0.0,Power,0.98633,0.01367,34.03,23.76,2021091201,63,7,...,242,Alex Highsmith,1,0,0,4.816638,6.835671,3.927467,2.438237,0.379183
2,202109120163,0.0,Power,0.98633,0.01367,34.03,23.76,2021091201,63,8,...,242,Alex Highsmith,1,0,0,4.790918,6.876053,3.927467,2.438237,0.379183
3,202109120163,0.0,Power,0.98633,0.01367,34.03,23.76,2021091201,63,9,...,242,Alex Highsmith,1,0,0,4.766424,6.936195,3.927467,2.438237,0.379183
4,202109120163,0.0,Power,0.98633,0.01367,34.03,23.76,2021091201,63,10,...,242,Alex Highsmith,1,0,0,4.738143,7.041023,3.927467,2.438237,0.379183
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
178513,20211021002175,1.0,Power,NaN,NaN,92.41,29.85,2021102100,2175,25,...,315,Jordan Elliott,3,0,0,5.114626,6.142353,4.666155,4.936801,-0.058002
178514,20211021002175,1.0,Power,NaN,NaN,92.41,29.85,2021102100,2175,26,...,315,Jordan Elliott,3,0,0,5.084191,6.162735,4.666155,4.936801,-0.058002
178515,20211021002175,1.0,Power,NaN,NaN,92.41,29.85,2021102100,2175,27,...,315,Jordan Elliott,3,0,0,5.049287,6.169935,4.666155,4.936801,-0.058002
178516,20211021002175,1.0,Power,NaN,NaN,92.41,29.85,2021102100,2175,28,...,315,Jordan Elliott,3,0,0,5.002799,6.168249,4.666155,4.936801,-0.058002


In [88]:
## Going to need to remove any power plays where this is an NA
model_data8 = model_data8.dropna(subset = ['pct_given_up'])

In [111]:
power_speed_perc = model_data7.groupby(['end_name', 'uniqueplayId', 'QB_Flip']).agg({
    'rushType':'first'
}).reset_index().groupby('end_name')['rushType'].agg(
    power_perc = lambda x: x[x == 'Power'].count()/x.count(),
    speed_perc = lambda x: x[x == 'Speed'].count()/x.count(),
    total = 'count'
).reset_index()

In [112]:
overall_perc_avs = model_data7.groupby(['uniqueplayId', 'QB_Flip']).agg({
    'rushType':'first'
}).reset_index()['rushType'].value_counts(normalize=True)
overall_perc_avs

Power    0.704846
Speed    0.295154
Name: rushType, dtype: float64

In [139]:
power_speed_perc['Av_Power'] = overall_perc_avs[0]
power_speed_perc['Av_Speed'] = overall_perc_avs[1]
power_speed_perc['num_plays'] = power_speed_perc['total'].sum()
power_speed_perc['end_power_perc'] = power_speed_perc['power_perc']*(power_speed_perc['total']/500) + power_speed_perc['Av_Power']*(1-power_speed_perc['total']/500)
power_speed_perc['end_speed_perc'] = power_speed_perc['speed_perc']*(power_speed_perc['total']/500) + power_speed_perc['Av_Speed']*(1-power_speed_perc['total']/500)

In [172]:
speed = speed.merge(power_speed_perc[['end_name', 'end_power_perc', 'end_speed_perc']], how='left', on='end_name')

In [173]:
speed['dist_moved'] = speed.groupby(['uniqueplayId','QB_Flip'])['Tackle_dis'].cumsum()

In [174]:
# Aggregate for grouped by to prepare data for modeling - one row per tackle/end instance
# The row should have the distance of the tackle in one second, and multiple predictors
# The predictors are speed/power percent of the end, weight and height of end, one-hot offensive linesman, obvious passing situation, and maybe quarter
def full_agg(grouped_df):
    row = grouped_df[grouped_df['frameId']==grouped_df['ball_snap_frame']+10]
    return pd.DataFrame({
    'one_sec_dist': row['dist_moved'],
    'end_power_perc': row['end_power_perc'],
    'end_speed_perc':  row['end_speed_perc'],
    'end_height': row['end_height'],
    'end_weight': row['end_weight'],
    'tackle_name': row['tackle_name'],
    'obvious_pass': row['obvious_pass'],
    'quarter': row['quarter']
})

speed_data = speed.groupby(['uniqueplayId', 'QB_Flip'], group_keys=True).apply(full_agg).reset_index().drop(['level_2'], axis=1)

In [175]:
speed_data['end_height'] = [int(j)*12+int(i) for j, i in speed_data['end_height'].str.split('-')]
speed_data['tackle_name'] = speed_data['tackle_name'].str.replace(" ", "_")
speed_data

,uniqueplayId,QB_Flip,one_sec_dist,end_power_perc,end_speed_perc,end_height,end_weight,tackle_name,obvious_pass,quarter
0,202109090097,1.0,1.46,0.699278,0.300722,75,245,Tristan_Wirfs,0,1
1,202109120288,0.0,1.36,0.698062,0.301938,75,275,Mekhi_Becton,0,1
2,202109120578,0.0,0.76,0.702890,0.297110,77,262,Laremy_Tunsil,0,1
3,202109120776,1.0,1.44,0.705348,0.294652,77,265,David_Quessenberry,1,1
4,202109121176,1.0,1.79,0.715515,0.284485,77,277,Ryan_Ramczyk,0,1
...,...,...,...,...,...,...,...,...,...,...
2610,20211025003506,0.0,0.87,0.708722,0.291278,76,279,Terron_Armstead,1,4
2611,20211025003536,0.0,1.84,0.696291,0.303709,76,260,Terron_Armstead,0,4
2612,20211025003536,1.0,1.98,0.704062,0.295938,76,330,Ryan_Ramczyk,0,4
2613,20211025003660,0.0,1.95,0.696291,0.303709,76,260,Terron_Armstead,1,4


In [186]:
speed_dummies = pd.get_dummies(speed_data, prefix="", prefix_sep='', columns=['tackle_name'])
speed_dummies['tackle_name'] = speed_data['tackle_name']
speed_dummies

,uniqueplayId,QB_Flip,one_sec_dist,end_power_perc,end_speed_perc,end_height,end_weight,obvious_pass,quarter,Alejandro_Villanueva,...,Trent_Williams,Trenton_Brown,Tristan_Wirfs,Ty_Sambrailo,Tyre_Phillips,Tyron_Smith,Yasir_Durant,Yodny_Cajuste,Yosuah_Nijman,tackle_name
0,202109090097,1.0,1.46,0.699278,0.300722,75,245,0,1,0,...,0,0,1,0,0,0,0,0,0,Tristan_Wirfs
1,202109120288,0.0,1.36,0.698062,0.301938,75,275,0,1,0,...,0,0,0,0,0,0,0,0,0,Mekhi_Becton
2,202109120578,0.0,0.76,0.702890,0.297110,77,262,0,1,0,...,0,0,0,0,0,0,0,0,0,Laremy_Tunsil
3,202109120776,1.0,1.44,0.705348,0.294652,77,265,1,1,0,...,0,0,0,0,0,0,0,0,0,David_Quessenberry
4,202109121176,1.0,1.79,0.715515,0.284485,77,277,0,1,0,...,0,0,0,0,0,0,0,0,0,Ryan_Ramczyk
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2610,20211025003506,0.0,0.87,0.708722,0.291278,76,279,1,4,0,...,0,0,0,0,0,0,0,0,0,Terron_Armstead
2611,20211025003536,0.0,1.84,0.696291,0.303709,76,260,0,4,0,...,0,0,0,0,0,0,0,0,0,Terron_Armstead
2612,20211025003536,1.0,1.98,0.704062,0.295938,76,330,0,4,0,...,0,0,0,0,0,0,0,0,0,Ryan_Ramczyk
2613,20211025003660,0.0,1.95,0.696291,0.303709,76,260,1,4,0,...,0,0,0,0,0,0,0,0,0,Terron_Armstead
